In [1]:
import pandas as pd
import networkx as nx
import numpy as np
from collections import OrderedDict
from tqdm import tqdm

In [15]:
df = pd.read_csv('wiki-topcats-reduced.txt', sep='\t', header = None)
df.columns = ['from', 'to']

In [24]:
#vertexes
fr = df.groupby('from').count()
to = df.groupby('to').count()
V = set(list(fr.index) + list(to.index))
print('Number of nodes =', len(V))

Number of nodes = 461193


In [29]:
#directed cause these two are different
print(df[df['from'] ==52], '\n')
print(df[df['from'] ==401135].head())

   from       to
0    52   401135
1    52  1069112
2    52  1163551 

          from      to
361754  401135   60219
361755  401135  167532
361756  401135  400980
361757  401135  401018
361758  401135  401019


In [30]:
print('Number of edges:',len(df.index))

Number of edges: 2645247


In [22]:
print('The average degree of the vertex is:', len(df.index)/len(V))

The average degree of the vertex is: 5.735661642739591


In [128]:
b = pd.DataFrame(d.index)
b.columns = ['from']

In [130]:
d = df.groupby('to').count()
a = pd.DataFrame(list(diff))
a.columns = ['from']
verteces = a.append(b)

In [5]:
o = open('wiki-topcats-categories.txt', 'r')
categories = {}
for line in o :
    line = line.replace('\n','')
    l = line.split(' ')
    l[0] = l[0].replace('Category:', '').replace(';','')
    try:
        #print(len(l[1:]))
        if len(l[1:])>=3500:
            categories[l[0]] = list(map(int, l[1:]))
    except:
        pass


In [25]:
input_category = list(categories.keys())[7]

In [ ]:
DG = nx.from_pandas_edgelist(df, 'from', 'to', create_using=nx.DiGraph )

In [8]:
class Dijkstra:

    def dijkstra(self, dic, dist):
        node = list(dic.keys())[0]
        actual_dist = list(dic.values())[0]
        l = []
        for i in list(self.graph[node]):
            try:
                if (i in self.visited):
                    if self.actual_node in self.visited[i]:
                        if (self.visited[i][1] > dist):
                            self.visited[i][1] = dist
                    else:
                        self.visited[i][self.actual_node] = dist
     
                if (self.unvisited[i] == -1):
                    del self.unvisited[i]
                    self.visited[i] = {self.actual_node:dist}
                l.append({i:dist})
            except:
                pass
                              
        for i in l:
            self.dijkstra(i, dist+1)
            
    
    
    def __init__(self, graph, categories, input_category):
        
        self.graph = graph
        self.nodes = categories
        self.initial = categories[input_category]
        self.unvisited = {}
        for i in graph.nodes:
            self.unvisited[i] = -1
        self.visited = {}
        
        for i in self.initial:
            try:
                self.actual_node = i
                self.visited[i] = {self.actual_node: 0}
                del self.unvisited[i]
                self.dijkstra({self.actual_node:0},1)
            except:
                pass
            


In [7]:
dij =Dijkstra(DG, categories, input_category)

In [26]:
median = {}
set_of_visited = set(list(dij.visited.keys()))
for i in categories:
    if i != input_category:
        shortest_path=[]
        s = set(categories[i]).intersection(set_of_visited)
        if len(s)> 0:
            for j in s:
                shortest_path += (list(dij.visited[j].values()))
            median[i] = np.median(shortest_path)
        else:
            median[i] = 100**100
median[input_category] = -1

In [27]:
block_ranking = OrderedDict()
block_ranking = OrderedDict(sorted(median.items(), key=lambda x: x[1]))

In [11]:
# map each article to the categories it is present

articles = {}
for name in categories:
    for article in categories[name]:
        if article in articles:
            articles[article].append(name)
        else:
            articles[article] = [name]

In [12]:
# if an article belongs to multiple categories, choose one according to block_ranking

for article in articles:
    if len(articles[article]) > 1:
        minimum = ''
        for cat in articles[article]:
            if (minimum == '') or (block_ranking[minimum] > block_ranking[cat]):
                minimum = cat
        articles[article] = [minimum]

In [13]:
categories_after_ranking = {}

for i in articles:
    if articles[i][0] in categories_after_ranking:
        categories_after_ranking[articles[i][0]] +=  [i]
    else:
        categories_after_ranking[articles[i][0]] =  [i]


In [14]:
# VERSIONE MODIFICATA
# da quella di prima cambia come trovare gli archi adatti (jj)

idx = 0
jj = nx.to_dict_of_lists(DG)

for name in tqdm(block_ranking):
    print(name)
    if idx == 0:
        weight_dict = {}
        boh = DG.subgraph(categories_after_ranking[name])
        weight_dict[name] = {}
        for i in (boh.in_degree):
            for j in jj[i[0]]:
                    DG[i[0]][j]['weight'] = i[1]
            idx +=1
        for  i in boh.in_degree:
            weight_dict[name][i[0]] = i[1]
    else:
        try:
            boh = DG.subgraph(categories_after_ranking[name])
            weight_dict[name] = {}

            for i in boh.in_degree:
                cumsum = i[1]
                for j in jj[i[0]]:
                        try:
                            cumsum+=(list(DG.edges[j,i[0]].values()))[0]
                        except:
                            pass
                weight_dict[name][i[0]] = cumsum


            for i in boh.in_degree:
                for j in jj[i[0]]:
                        DG[i[0]][j]['weight'] = weight_dict[name][i[0]]
        except:
            pass

weight_dict

  0%|                                                                                           | 0/35 [00:00<?, ?it/s]

Year_of_birth_unknown
Year_of_birth_missing


  6%|████▋                                                                              | 2/35 [00:00<00:02, 12.12it/s]

Article_Feedback_Pilot


  9%|███████                                                                            | 3/35 [00:01<00:10,  3.08it/s]

...

Association_football_midfielders
Association_football_defenders
Indian_films


 97%|███████████████████████████████████████████████████████████████████████████████▋  | 34/35 [00:20<00:00,  1.02it/s]

Major_League_Baseball_pitchers


100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [00:20<00:00,  1.70it/s]


{'Year_of_birth_unknown': {958476: 1,
  958480: 3,
  172050: 3,
  368658: 0,
  368661: 0,
  213014: 1,
  172074: 2,
  1130540: 0,
  360497: 0,
  1269822: 0,
  1204299: 1,
  1204300: 0,
  65614: 2,
  360531: 0,
  172116: 0,
  1359957: 0,
  1081428: 0,
  1171545: 0,
  1122394: 2,
  360545: 1,
  1253475: 0,
  360549: 1,
  827516: 0,
  958589: 0,
  958595: 0,
  1319047: 1,
  1319048: 3,
  1605768: 0,
  1171594: 0,
  1663112: 2,
  1663113: 1,
  1663114: 2,
  360595: 3,
  114842: 0,
  360604: 1,
  401579: 0,
  245933: 2,
  1122479: 1,
  1261744: 0,
  827571: 0,
  1147068: 0,
  819393: 0,
  114883: 0,
  704707: 0,
  1589450: 0,
  1122515: 0,
  1261789: 0,
  1122526: 2,
  1122532: 0,
  188650: 0,
  1417470: 0,
  1753342: 0,
  1417472: 0,
  1753343: 1,
  1491210: 0,
  1261835: 1,
  295185: 0,
  1081633: 0,
  1417505: 0,
  1122603: 3,
  1122605: 3,
  1270073: 0,
  1171770: 0,
  1614149: 0,
  1122629: 2,
  532809: 0,
  155980: 0,
  450892: 0,
  1122637: 1,
  65872: 0,
  1122640: 2,
  426344: 0,
 

In [ ]:
# VERSIONE VISTA INSIEME

idx = 0
for name in tqdm(block_ranking):
    print(name)
    if idx == 0:
        weight_dict = {}
        boh = DG.subgraph(categories_after_ranking[name])
        weight_dict[name] = {}
        for i in boh.in_degree:
            print(idx)
            for j in DG.edges:
                if j[0] == i[0]:
                    DG[i[0]][j[1]]['weight'] = i[1]
            idx +=1
        for i in boh.in_degree: 
            weight_dict[name][i[0]] = i[1]
        

        try:
            boh = DG.subgraph(categories_after_ranking[name])
            weight_dict[name] = {}

            for i in boh.in_degree:
                cumsum = i[1]
                for j in DG.edges:
                    if j[1] == i[0]:
                        try:
                            cumsum+=(list(DG.edges[j[0],j[1]].values()))[0]
                        except:
                            pass
                weight_dict[name][i[0]] = cumsum


            for i in boh.in_degree:
                for j in DG.edges:
                    if j[0] == i[0]:
                        DG[i[0]][j[1]]['weight'] = weight_dict[name][i[0]]
        except:
            pass

weight_dict









  0%|                                                                                           | 0/35 [00:00<?, ?it/s]

Year_of_birth_unknown
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
